# 6. Advanced RAG


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 6.2. ハンズオンの準備


In [2]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 10.9 MB/s eta 0:00:00
   

In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

385


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。'

## 6.3. 検索クエリの工夫


### HyDE（Hypothetical Document Embeddings）


In [6]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [7]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、開発者がAIアプリケーションを構築する際に、さまざまなコンポーネントを簡単に組み合わせて利用できるようにするためのPythonパッケージおよび企業です。元々はオープンソースの単一パッケージとして始まりましたが、現在では広範なエコシステムに進化しています。\n\nLangChainの主な特徴には以下が含まれます：\n\n1. **標準化されたコンポーネントインターフェース**: 多様なAIモデルや関連コンポーネントのAPIの違いを克服するため、LangChainは共通のインターフェースを提供し、プロバイダー間の切り替えを容易にします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを効率的に接続し、複雑なアプリケーションを構築するための制御フローをサポートします。\n\n3. **可観測性と評価**: アプリケーションの複雑さが増す中で、開発者がアプリケーションの動作を理解し、迅速に評価を行うためのツールを提供します。\n\nLangChainは、チャットモデルやLLM（大規模言語モデル）、ベクトルストアなど、さまざまなコンポーネントをサポートしており、開発者は自分のユースケースに最適なコンポーネントを選択して使用できます。また、LangGraphというライブラリを使用することで、複雑なアプリケーションのオーケストレーションが可能になります。さらに、LangSmithというプラットフォームを通じて、アプリケーションの可観測性や評価を行うことができます。'

### 複数の検索クエリの生成


In [8]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [9]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。\n\n### 主な特徴\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。特に、LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **運用**: LangSmithを使用してアプリケーションを監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\n### エコシステム\nLangChainは、さまざまなモデルや関連コンポーネントの標準化されたインターフェースを提供し、開発者が異なるプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりできるようにします。また、アプリケーションの複雑さが増すにつれて、オーケストレーションや可視化、評価のニーズが高まることに対応しています。\n\n### まとめ\nLangChainは、AIアプリケーションの開発を容易にし、開発者が自分のユースケースに最適なコンポーネントを選択できる柔軟性を提供します。'

## 6.4. 検索後の工夫


### RAG Fusion


In [10]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [11]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainの目的は、開発者が推論を行うアプリケーションを簡単に構築できるようにすることです。もともとはオープンソースのパッケージとして始まりましたが、現在は企業とエコシステム全体に進化しています。\n\nLangChainは以下の主要な機能を提供しています：\n\n1. **標準化されたコンポーネントインターフェース**: 様々なAIアプリケーションのモデルや関連コンポーネントの多様性に対処するため、LangChainは主要コンポーネントの標準インターフェースを提供し、プロバイダー間の切り替えを容易にします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを組み合わせて複雑なアプリケーションを構築するための効率的な接続をサポートします。\n\n3. **可観測性と評価**: アプリケーションが複雑になるにつれて、内部で何が起こっているのかを理解することが難しくなるため、開発者がアプリケーションを監視し、迅速に評価できるようにします。\n\nLangChainは、開発、運用、デプロイの各段階で役立つオープンソースのコンポーネントやサードパーティの統合を提供し、特にLangGraphを使用して状態を持つエージェントを構築することができます。また、LangSmithを使用してアプリケーションを監視し、評価することも可能です。'

### Cohere のリランクモデルを使用する準備


In [12]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

In [13]:
!pip install langchain-cohere==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0


### Cohere のリランクモデルの導入


In [14]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準化されたインターフェースを提供することで、開発者が異なるコンポーネントを簡単に切り替えたり、組み合わせたりできるようにします。また、複雑なアプリケーションのオーケストレーションをサポートするために、LangGraphというライブラリも提供しています。これにより、開発者はアプリケーションのフローをノードとエッジのセットとして表現し、複雑な制御フローを管理できます。'

## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [15]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [16]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [17]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {retriever}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [18]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。'

In [19]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日、2024年11月19日(火)の天気は「晴時々曇」で、最高気温は13℃、最低気温は8℃です。降水確率は0％で、北の風が吹く予報です。'

### ハイブリッド検索の実装


In [20]:
!pip install rank-bm25==0.2.2

In [21]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [22]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [23]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、PythonとJavaScriptの両方のライブラリが提供されており、さまざまなチュートリアルやガイドが用意されています。'

In [24]:
# prompt: langchainの概要を教えて

このコードは、LangChain を使用して、与えられた質問に対する回答を生成する様々なRAG（Retrieval Augmented Generation）手法を例示しています。LangChainのドキュメントをベクトルデータベース（Chroma）に格納し、質問に対して関連するドキュメントを検索し、それらを用いてLLM（ここではGPT-4o-mini）が回答を生成します。

以下に、コードの主要な部分と、それぞれの手法の概要を説明します。

**1. 基本的なRAG:**

* `retriever = db.as_retriever()`: Chromaデータベースから関連ドキュメントを取得するためのRetrieverを作成します。
* `chain`: 質問を受け取り、Retrieverで関連ドキュメントを検索し、プロンプトテンプレートに埋め込み、LLMに渡して回答を生成するチェーンです。これは最も基本的なRAG手法です。

**2. 検索クエリの工夫:**

* **HyDE (Hypothetical Document Embeddings):** 質問から仮説的なドキュメントを生成し、そのドキュメントを用いて検索を行う手法です。質問の意図をより正確に捉え、関連性の高いドキュメントを検索できる可能性があります。
* **複数の検索クエリの生成:**  一つの質問から複数の検索クエリを生成し、それぞれのクエリで検索を実行することで、多様な視点から情報を取得し、より包括的な回答を生成します。

**3. 検索後の工夫:**

* **RAG Fusion:** 複数の検索結果を統合する手法です。このコードでは、Reciprocal Rank Fusionを使用し、複数の検索結果におけるドキュメントのランクに基づいて、各ドキュメントの重要度を計算し、最終的なランキングを生成しています。重複した情報を減らし、より関連性の高いドキュメントを上位に表示することを目指します。
* **Cohereのリランクモデル:**  Cohereのrerankモデルを用いて、検索結果を再ランク付けすることで、LLMへの入力としてより適切なドキュメントを選択し、回答の品質向上を図ります。

**4. 複数のRetrieverを使う工夫:**

* **LLMによるルーティング:** 質問の内容に基づいて、適切なRetrieverを選択する手法です。このコードでは、LangChainのドキュメントを対象とするRetrieverと、Web検索を行うRetrieverを準備し、LLMが質問内容に応じてどちらのRetrieverを使用するかを決定します。
* **ハイブリッド検索:** 複数のRetriever（このコードではChromaとBM25）を併用し、それぞれの強みを活かして検索を行う手法です。


**LangChainの概要:**

LangChainは、大規模言語モデル（LLM）をアプリケーションに統合するためのフレームワークです。様々なLLMやデータソースを繋ぎ、複雑な処理を簡単に実現できます。このコードは、その中でもRAGに焦点を当て、様々な手法を用いて、より高品質な回答を生成する例を示しています。

**コードの実行には以下の環境変数が設定されている必要があります。**

* `OPENAI_API_KEY`
* `LANGCHAIN_TRACING_V2`
* `LANGCHAIN_ENDPOINT`
* `LANGCHAIN_API_KEY`
* `LANGCHAIN_PROJECT`
* `TAVILY_API_KEY`
* `COHERE_API_KEY`


このコードは、LangChainの様々な機能を組み合わせることで、高度なRAGシステムを構築する方法を示しており、LangChainの柔軟性と拡張性を示す良い例となっています。
